## 1. Importing Data
Let's import them

In [ ]:
# Data manipulation
import numpy as np
import pandas as pd

# Cross-validation
from sklearn.model_selection import train_test_split

# Visualization
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

# Modeling
import tensorflow as tf
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam

# Randomization
import random

## 2. Setting Parameters and My Beautiful Helper Functions

In [ ]:
IMG_SIZE = 224
BATCH_SIZE = 32

# View an image
def view_random_image():
    sample = train.sample(n=1)
    target_file = sample['file_path'].values[0]

  # Read in the image and plot it using matplotlib
    img = mpimg.imread(target_file)
    print(img.shape)
    plt.imshow(img)
    plt.title(sample['Pawpularity'].values[0])
    plt.axis("off");
    
def plot_model(history):
    plt.figure(figsize=(20, 6))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['rmse'], label='training rmse')
    plt.plot(history.history['val_rmse'], label='test rmse')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='training loss')
    plt.plot(history.history['val_loss'], label='test loss')
    plt.legend()
    
def preprocess(X_train, y_train):
    image_string = tf.io.read_file(X_train)
    image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    return image, tf.cast(y_train, tf.float32)

def rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean((y_true -  y_pred) ** 2))

def final_preprocess(file_path):
    image_string = tf.io.read_file(file_path)
    image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    return image

## 3. Downloading and Preperation Dataset

In [ ]:
# Reading Data
train = pd.read_csv("../input/petfinder-pawpularity-score/train.csv")
test = pd.read_csv("../input/petfinder-pawpularity-score/test.csv")
sample_submission = pd.read_csv("../input/petfinder-pawpularity-score/sample_submission.csv")

In [ ]:
train["file_path"] = train["Id"].apply(lambda identifier: "../input/petfinder-pawpularity-score/train/" + identifier + ".jpg")
test["file_path"] = test["Id"].apply(lambda identifier: "../input/petfinder-pawpularity-score/test/" + identifier + ".jpg")
train['Pawpularity'] = train['Pawpularity'] / 100.0
train.shape, test.shape

In [ ]:
view_random_image()

In [ ]:
sub_data = train.sample(frac=1, random_state=42)
sub_data.shape

In [ ]:
X = sub_data['file_path']
y = sub_data['Pawpularity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).map(preprocess).batch(BATCH_SIZE)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).map(preprocess).batch(BATCH_SIZE)

## 4. Cooking Some Code

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(
        min_delta=1e-4, 
        patience=10
    )
    
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
        factor=0.3,
        patience=2, 
        min_lr=1e-7
    )
    
callbacks = [early_stop, reduce_lr]

In [ ]:
if 'efficientnetb0' not in globals():
    efficientnetb0 = tf.keras.models.load_model('../input/efficientnet-b0')

In [ ]:
efficientnetb0.trainable = False
efficientnetb0.trainable

In [ ]:
tf.keras.backend.clear_session()

seed = 42
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

# Create our own model
inputs = Input(shape=(IMG_SIZE, IMG_SIZE, 3), name='input_layer')
x = efficientnetb0(inputs)
output =  Dense(1, activation='sigmoid', name='output_layer')(x)
efficientnetb0_model = Model(inputs=[inputs], outputs=[output])

efficientnetb0_model.compile(loss=BinaryCrossentropy(),
                     optimizer=Adam(),
                     metrics=[rmse])

history_efficientnetb0_model = efficientnetb0_model.fit(
    train_dataset, 
    epochs=2,
    steps_per_epoch=len(train_dataset),
    validation_data=test_dataset,
    validation_steps=len(test_dataset),
    callbacks=callbacks)

## 5. The Part I like Most: Submission

In [ ]:
final = tf.data.Dataset.from_tensor_slices((test['file_path'])).map(final_preprocess).batch(BATCH_SIZE)
y_pred = efficientnetb0_model.predict(final) * 100
submission = pd.DataFrame({'Id':test['Id'],'Pawpularity': y_pred.flatten()}).reset_index(drop=True)
submission.head()
submission.to_csv('submission.csv', index=False)